### Cine21 영화배우 정보 웹스크래핑
1. Site 주소 : http://www.cine21.com/rank/person/
2. Request URL : http://www.cine21.com/rank/person/content   
  - 크롬 개발자 도구의 network 탭에서 xhr 인 것만 필터링 
3. Request Method : POST
4. Form Data 
 - section: actor
 - period_start: 2021-11
 - gender: all
 - page: 1
5. 2년치 데이터를 한꺼번에 스크래핑 해서 actors.json 파일로 저장 

### 사용된 라이브러리에 대한 설명
1. urllib(내부), requests(외부) : Http Client 
2. BeautifulSoup(외부) : html, xml Parser  
  : 원하는 html tag를 선택하는 함수 select(css선택자), select_one()  
3. re(내부) : regular expression  

In [1]:
import requests
from bs4 import BeautifulSoup
import re
from urllib.parse import urljoin

### 1단계 
* 1 page 7명의 이름과 상세정보 추출

### 정규표현식 (regular expression)
* \w는 word를 표현하며 알파벳 + 숫자 + _ 중의 한 문자임을 의미한다.
* 문자가 출현한 횟수를 나타낼때 
 <pre>   
  * : 0 ~ n,  + : 1 ~ n , ? : 0(zero) or 1 
 </pre> 
* 정규표현식에 특수문자를 사용할때는 특수문자 앞에 \(back slash)를 반드시 주어야 한다
* .(period)는 줄바꿈 문자(\n)을 제외한 모든 글자 1개를 의미한다
* Greedy vs Non-Greedy
 - Greedy 욕심많은 (.*) : 모든 태그가 다 선택
 - Non-Greedy 욕심없는 (.*?) : 첫번째 매칭되는 태그만 선택
* [\n\r\t] : \n,\r,\t 중에 하나만 찾는다.
 - re.compile('[\n\r\t]')는 해당 정규표현식을 찾아주는 컴파일러를 생성한다.
 - compile 함수내의 정규표현식 앞에는 항상 r 을 붙혀 주어야 한다.

In [8]:
req_url = 'http://www.cine21.com/rank/person/content'
req_param = {
    'section': 'actor',
    'period_start': '2021-11',
    'gender': 'all',
    'page': 1
}

# url로 post방식으로 요청 보내기
res = requests.post(req_url, data=req_param)
print(res.status_code)
if res.ok: #status_code == 200
    # BeautifulSoup
    soup = BeautifulSoup(res.text, 'html.parser')
    
    for li_tag in soup.select('li.people_li'):
        div_tag = li_tag.select_one('div.name')
        #상세페이지 url 추출
        detail_url = div_tag.select_one('a')['href']
        detail_url = urljoin(req_url,detail_url)
        
        name = div_tag.text
        name_result = re.sub('\(\w*\)','',name)
        print(name_result, detail_url)
                       
        res = requests.get(detail_url)
        if res.ok:
            soup = BeautifulSoup(res.text,'html.parser')
            for li_tag in soup.select('ul.default_info li'):
                actor_dict_key = li_tag.select_one('span.tit').text
                #print(li_tag)
                #<li><span class="tit">직업</span>배우</li> => <li>배우</li>
                li_result = re.sub('<span.*>.*<\/span>','',str(li_tag))
                #<li>배우</li> => 배우
                actor_dict_value = re.sub('<.*?>','',li_result)
                regex = re.compile(r'[\n\r\t]')
                actor_dict_value = regex.sub(' ',actor_dict_value)        
                print(actor_dict_key,actor_dict_value)
        print('===========================')


200
윤계상 http://www.cine21.com/db/person/info/?person_id=19889
다른 이름 지오디;god
직업 배우
생년월일 1978-12-20
성별 남
홈페이지 
https://www.facebook.com/saram.yoonkyesang
https://www.instagram.com/kyesang78/

신장/체중 182cm, 62kg
학교 경희대학교 포스트모던학과 휴학
취미 컴퓨터게임, 스노우보드
특기 표정연기, 춤추기
박용우 http://www.cine21.com/db/person/info/?person_id=3984
직업 배우
생년월일 1971-03-16
성별 남
신장/체중 176cm, 65kg
학교 중앙대학교 연극영화과
취미 볼링 , 피아노
특기 노래, 춤, 태권도
소속사 스타파크엔터테인먼트
손석구 http://www.cine21.com/db/person/info/?person_id=104968
직업 배우
생년월일 1983-00-00
성별 남
임지연 http://www.cine21.com/db/person/info/?person_id=471
직업 배우
성별 여
홈페이지 
http://www.artistcompany.co.kr/portfolio-item/im-ji-yeon/

소속사 아티스트컴퍼니
공민정 http://www.cine21.com/db/person/info/?person_id=103208
다른 이름 김민정
직업 배우
생년월일 1986-09-30
성별 여
박지환 http://www.cine21.com/db/person/info/?person_id=35760
직업 배우
생년월일 1980-09-05
성별 남
류승룡 http://www.cine21.com/db/person/info/?person_id=39673
다른 이름 류승용; 유승룡
직업 배우
생년월일 1970-11-29
성별 남
신장/체중 175cm, 72kg
학교 서울예술대학 연극
소속사 필름있수다


### 2단계 
* 1 page 7명의 이름,흥행지수,랭킹과 상세정보 추출
* 배우의 정보는 dict에 저장하고, dict를 list에 저장한다.

In [17]:
req_url = 'http://www.cine21.com/rank/person/content'
req_param = {
    'section': 'actor',
    'period_start': '2021-11',
    'gender': 'all',
    'page': 1
}

# url로 post방식으로 요청 보내기
res = requests.post(req_url, data=req_param)
print(res.status_code)
if res.ok: #status_code == 200
    # BeautifulSoup
    soup = BeautifulSoup(res.text, 'html.parser')
    
    #7명 actor들의 상세정보를 저장할 list 선언
    actor_detail_list = []
    
    for li_tag in soup.select('li.people_li'):
        #actor 1명의 상세정보를 저장할 dict를 선언
        actor_detail_dict = {}
        
        div_tag = li_tag.select_one('div.name')
        #상세페이지 url 추출
        detail_url = div_tag.select_one('a')['href']
        detail_url = urljoin(req_url,detail_url)
        
        #actor이름을 추출
        name = div_tag.text
        #윤계상(2편) => 윤계상
        name_result = re.sub('\(\w*\)','',name)
        actor_detail_dict['배우이름'] = name_result
        
        #actor랭킹를 추출
        ranking = li_tag.select_one('span.grade').text
        actor_detail_dict['랭킹'] = int(ranking)
        
        #흥행지수 추출
        hit = li_tag.select_one('ul.num_info > li > strong').text
        actor_detail_dict['흥행지수'] = int(hit.replace(',',''))
        
        #상세정보 페이지에 요청을 보내기
        res = requests.get(detail_url)
        if res.ok:
            soup = BeautifulSoup(res.text,'html.parser')
            
            #기본정보 
            for li_tag in soup.select('ul.default_info li'):
                actor_dict_key = li_tag.select_one('span.tit').text
                #print(li_tag)
                #<li><span class="tit">직업</span>배우</li> => <li>배우</li>
                li_result = re.sub('<span.*>.*<\/span>','',str(li_tag))
                #<li>배우</li> => 배우
                actor_dict_value = re.sub('<.*?>','',li_result)
                regex = re.compile(r'[\n\r\t]')
                actor_dict_value = regex.sub(' ',actor_dict_value)
                
                actor_detail_dict[actor_dict_key] = actor_dict_value
            
            #출연영화
            #출연영화목록을 저장할 리스트 선언
            movie_title_list = []
            for movie_li_tag in soup.select('ul.part_works li'):
                movie_title = movie_li_tag.select_one('span.tit').text
                movie_title_list.append(movie_title)                
            actor_detail_dict['출연영화'] = movie_title_list
            
            #actor들의 상세정보 dict를 list에 추가하기
            actor_detail_list.append(actor_detail_dict)

print(len(actor_detail_list))
print(actor_detail_list)        

200
7
[{'배우이름': '윤계상', '랭킹': 1, '흥행지수': 69585, '다른 이름': '지오디;god', '직업': '배우', '생년월일': '1978-12-20', '성별': '남', '홈페이지': ' https://www.facebook.com/saram.yoonkyesang https://www.instagram.com/kyesang78/ ', '신장/체중': '182cm, 62kg', '학교': '경희대학교 포스트모던학과 휴학', '취미': '컴퓨터게임, 스노우보드', '특기': '표정연기, 춤추기', '출연영화': ['크라임 퍼즐', '유체이탈자', '초콜릿']}, {'배우이름': '박용우', '랭킹': 2, '흥행지수': 60885, '직업': '배우', '생년월일': '1971-03-16', '성별': '남', '신장/체중': '176cm, 65kg', '학교': '중앙대학교 연극영화과', '취미': '볼링 , 피아노', '특기': '노래, 춤, 태권도', '소속사': '스타파크엔터테인먼트', '출연영화': ['유체이탈자', '드라마 스테이지 - 오우거', '카센타']}, {'배우이름': '손석구', '랭킹': 3, '흥행지수': 55320, '직업': '배우', '생년월일': '1983-00-00', '성별': '남', '출연영화': ['연애 빠진 로맨스', '언프레임드', 'D.P.']}, {'배우이름': '임지연', '랭킹': 4, '흥행지수': 52188, '직업': '배우', '성별': '여', '홈페이지': ' http://www.artistcompany.co.kr/portfolio-item/im-ji-yeon/ ', '소속사': '아티스트컴퍼니', '출연영화': ['손현주의 간이역', '유체이탈자', '타짜: 원 아이드 잭']}, {'배우이름': '공민정', '랭킹': 5, '흥행지수': 47593, '다른 이름': '김민정', '직업': '배우', '생년월일': '1986-09-30', '성별': '여', '출연영화':

### 3단계 
* 1,2,3 page 21명의 이름,흥행지수,랭킹과 상세정보 추출
* 배우의 정보는 dict에 저장하고, dict를 list에 저장한다.

In [19]:
req_url = 'http://www.cine21.com/rank/person/content'
req_param = {
    'section': 'actor',
    'period_start': '2021-11',
    'gender': 'all',
}

#21명 actor들의 상세정보를 저장할 list 선언
actor_detail_list = []

for page_num in range(1,4):
    req_param['page'] = page_num
    print(f'---> {page_num} 페이지 출력됨')
    
    # url로 post방식으로 요청 보내기
    res = requests.post(req_url, data=req_param)
    print(res.status_code)
    if res.ok: #status_code == 200
        # BeautifulSoup
        soup = BeautifulSoup(res.text, 'html.parser')


        for li_tag in soup.select('li.people_li'):
            #actor 1명의 상세정보를 저장할 dict를 선언
            actor_detail_dict = {}

            div_tag = li_tag.select_one('div.name')
            #상세페이지 url 추출
            detail_url = div_tag.select_one('a')['href']
            detail_url = urljoin(req_url,detail_url)

            #actor이름을 추출
            name = div_tag.text
            #윤계상(2편) => 윤계상
            name_result = re.sub('\(\w*\)','',name)
            actor_detail_dict['배우이름'] = name_result

            #actor랭킹를 추출
            ranking = li_tag.select_one('span.grade').text
            actor_detail_dict['랭킹'] = int(ranking)

            #흥행지수 추출
            hit = li_tag.select_one('ul.num_info > li > strong').text
            actor_detail_dict['흥행지수'] = int(hit.replace(',',''))

            #상세정보 페이지에 요청을 보내기
            res = requests.get(detail_url)
            if res.ok:
                soup = BeautifulSoup(res.text,'html.parser')

                #기본정보 
                for li_tag in soup.select('ul.default_info li'):
                    actor_dict_key = li_tag.select_one('span.tit').text
                    #print(li_tag)
                    #<li><span class="tit">직업</span>배우</li> => <li>배우</li>
                    li_result = re.sub('<span.*>.*<\/span>','',str(li_tag))
                    #<li>배우</li> => 배우
                    actor_dict_value = re.sub('<.*?>','',li_result)
                    regex = re.compile(r'[\n\r\t]')
                    actor_dict_value = regex.sub(' ',actor_dict_value)

                    actor_detail_dict[actor_dict_key] = actor_dict_value

                #출연영화
                #출연영화목록을 저장할 리스트 선언
                movie_title_list = []
                for movie_li_tag in soup.select('ul.part_works li'):
                    movie_title = movie_li_tag.select_one('span.tit').text
                    movie_title_list.append(movie_title)                
                actor_detail_dict['출연영화'] = movie_title_list

                #actor들의 상세정보 dict를 list에 추가하기
                actor_detail_list.append(actor_detail_dict)

print(len(actor_detail_list))
print(actor_detail_list)

---> 1 페이지 출력됨
200
---> 2 페이지 출력됨
200
---> 3 페이지 출력됨
200
21
[{'배우이름': '윤계상', '랭킹': 1, '흥행지수': 69585, '다른 이름': '지오디;god', '직업': '배우', '생년월일': '1978-12-20', '성별': '남', '홈페이지': ' https://www.facebook.com/saram.yoonkyesang https://www.instagram.com/kyesang78/ ', '신장/체중': '182cm, 62kg', '학교': '경희대학교 포스트모던학과 휴학', '취미': '컴퓨터게임, 스노우보드', '특기': '표정연기, 춤추기', '출연영화': ['크라임 퍼즐', '유체이탈자', '초콜릿']}, {'배우이름': '박용우', '랭킹': 2, '흥행지수': 60885, '직업': '배우', '생년월일': '1971-03-16', '성별': '남', '신장/체중': '176cm, 65kg', '학교': '중앙대학교 연극영화과', '취미': '볼링 , 피아노', '특기': '노래, 춤, 태권도', '소속사': '스타파크엔터테인먼트', '출연영화': ['유체이탈자', '드라마 스테이지 - 오우거', '카센타']}, {'배우이름': '손석구', '랭킹': 3, '흥행지수': 55320, '직업': '배우', '생년월일': '1983-00-00', '성별': '남', '출연영화': ['연애 빠진 로맨스', '언프레임드', 'D.P.']}, {'배우이름': '임지연', '랭킹': 4, '흥행지수': 52188, '직업': '배우', '성별': '여', '홈페이지': ' http://www.artistcompany.co.kr/portfolio-item/im-ji-yeon/ ', '소속사': '아티스트컴퍼니', '출연영화': ['손현주의 간이역', '유체이탈자', '타짜: 원 아이드 잭']}, {'배우이름': '공민정', '랭킹': 5, '흥행지수': 47593, '다른 이름': '김민정'

### 4단계 
* 모든 page의 배우 이름,흥행지수,랭킹과 상세정보 추출
* 배우의 정보는 dict에 저장하고, dict를 list에 저장한다.
* itertools 모듈의 count() 함수를 사용하여 무한 Looping
 - 무한 Loop를 탈출할 수 있는 조건
 - actor_li_tag = soup.select('li.people_li') 의 actor_list_tag의 길이가 0 이면 탈출

In [ ]:
from itertools import count

req_url = 'http://www.cine21.com/rank/person/content'
req_param = {
    'section': 'actor',
    'period_start': '2021-11',
    'gender': 'all',
}

#21명 actor들의 상세정보를 저장할 list 선언
actor_detail_list = []

#무한looping
for page_num in count(1):
    req_param['page'] = page_num
    
    if page_num % 10 == 0:
        print(f'---> {page_num} 페이지 출력됨')
    
    # url로 post방식으로 요청 보내기
    res = requests.post(req_url, data=req_param)
    print(res.status_code)
    if res.ok: #status_code == 200
        # BeautifulSoup
        soup = BeautifulSoup(res.text, 'html.parser')
        
        #해당 page가 없으면 li tag가 없으므로 무한루프를 탈출
        actor_li_tags = soup.select('li.people_li')
        if len(actor_li_tags) == 0:
            break
        
        for li_tag in actor_li_tags:
            #actor 1명의 상세정보를 저장할 dict를 선언
            actor_detail_dict = {}

            div_tag = li_tag.select_one('div.name')
            #상세페이지 url 추출
            detail_url = div_tag.select_one('a')['href']
            detail_url = urljoin(req_url,detail_url)

            #actor이름을 추출
            name = div_tag.text
            #윤계상(2편) => 윤계상
            name_result = re.sub('\(\w*\)','',name)
            actor_detail_dict['배우이름'] = name_result

            #actor랭킹를 추출
            ranking = li_tag.select_one('span.grade').text
            actor_detail_dict['랭킹'] = int(ranking)

            #흥행지수 추출
            hit = li_tag.select_one('ul.num_info > li > strong').text
            actor_detail_dict['흥행지수'] = int(hit.replace(',',''))

            #상세정보 페이지에 요청을 보내기
            res = requests.get(detail_url)
            if res.ok:
                soup = BeautifulSoup(res.text,'html.parser')

                #기본정보 
                for li_tag in soup.select('ul.default_info li'):
                    actor_dict_key = li_tag.select_one('span.tit').text
                    #print(li_tag)
                    #<li><span class="tit">직업</span>배우</li> => <li>배우</li>
                    li_result = re.sub('<span.*>.*<\/span>','',str(li_tag))
                    #<li>배우</li> => 배우
                    actor_dict_value = re.sub('<.*?>','',li_result)
                    regex = re.compile(r'[\n\r\t]')
                    actor_dict_value = regex.sub(' ',actor_dict_value)

                    actor_detail_dict[actor_dict_key] = actor_dict_value

                #출연영화
                #출연영화목록을 저장할 리스트 선언
                movie_title_list = []
                for movie_li_tag in soup.select('ul.part_works li'):
                    movie_title = movie_li_tag.select_one('span.tit').text
                    movie_title_list.append(movie_title)                
                actor_detail_dict['출연영화'] = movie_title_list

                #actor들의 상세정보 dict를 list에 추가하기
                actor_detail_list.append(actor_detail_dict)

print(len(actor_detail_list))
print(actor_detail_list)

### 5단계
* actor_detail_final_list를 actors.json 파일로 저장하기